In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Compare 3T model with GaInP/GaAs r-type data
## Example MS874 from Geisz, et al., Cell Reports Physical Science 2, p. 100677 (2021) 

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from pvcircuit.PlotWithControls import PlotsWithControls
import pvcircuit as pvc
import ipywidgets as widgets
#%matplotlib inline
#import ipympl
%matplotlib widget

In [7]:
A=0.25
Aratio=0.89
# data iv3T vs V
path = pvc.notebook_datapath
fileA = 'MS874n4papy_C_CZ_JA.csv'
fileB = 'MS874n4papy_C_CZ_JB.csv'
MS874_V_dataiv = pvc.IV3T.from_csv('MS874_V_dataiv',path,fileA,fileB,'V','CZ', area=A) #Iscale=1000./A)
# data iv3T vs I
fileA = 'MS874n4papy_C_CZ_VA.csv'
fileB = 'MS874n4papy_C_CZ_VB.csv'
MS874_I_dataiv = pvc.IV3T.from_csv('MS874_I_dataiv',path,fileA,fileB,'I','CZ', area=A) #Iscale=1000./A)
#dark data
fileA = 'MS874n4Cdark_CZ_JA.csv'
fileB = 'MS874n4Cdark_CZ_JB.csv'
MS874dark_dataiv = pvc.IV3T.from_csv('MS874dark_dataiv',path,fileA,fileB,'V','CZ', area=A) #Iscale=1000./A)

In [8]:
# model
MS874dev3T = pvc.Tandem3T()

MS874dev3T.set(name='MS874dev3T', Rz = 0.3)
MS874dev3T.top.set(Eg=1.861, n=[1.0, 1.8], J0ratio=[158, 12], Jext=0.01525, Gsh=1e-12)
#MS874dev3T.top.set(Rser=1.1, beta=2e-6, area=A)
MS874dev3T.top.set(Rser=1.1, beta=3e-6, area=A)

MS874dev3T.bot.set(Eg=1.421, n=[1.0, 2.0], J0ratio=[60., 5.], Jext=0.0113/Aratio, Gsh=1e-10, pn=-1)
#MS874dev3T.bot.set(Rser=0.6, beta=15, area=A*Aratio)
MS874dev3T.bot.set(Rser=0.6, beta=24, area=A*Aratio)
MS874dev3T.bot.RBB_dict={'method':'JFG', 'mrb':7., 'J0rb':8e-7, 'Vrb':0.}

In [12]:
# ui, Vax, Iax = MS874dev3T.controls(MS874_V_dataiv, MS874_I_dataiv, MS874dark_dataiv, hex=False)
# ui

pwc = PlotsWithControls(MS874dev3T,MS874_V_dataiv, MS874_I_dataiv, MS874dark_dataiv, hex=False)
ui = pwc.ui
ui

In [10]:
# plot dark currents
plt.ioff()
Tax, Tobj = MS874dark_dataiv.plot(zkey='Ito', log=True, cmap='turbo')
Bax, Robj = MS874dark_dataiv.plot(zkey='Iro', log=True, cmap='turbo')
Lout = widgets.Output()
Rout = widgets.Output()
ToutBox = widgets.HBox([Lout, Rout])
with Lout:
    Tax.get_figure().show()
with Rout:
    Bax.get_figure().show()
plt.ion()
ToutBox

In [11]:
VM32, MPP32 = MS874dev3T.VM(3,2)
VM43, MPP43 = MS874dev3T.VM(4,3)
CM, MPPCM = MS874dev3T.CM()
sp = MS874dev3T.specialpoints()
sp.append(MPP32)
sp.append(MPP43)
sp.append(MPPCM)
sp

MS874dev3T SpecialPoints: <pvcircuit.iv3T.IV3T class>
{'name': 'MS874dev3T SpecialPoints', 'meastype': 'CZ', 'area': 0.25, 'shape': (9,)}
sizes(9, 9)

      #          mIro     mIzo     mIto     Vzt      Vrz      Vtr      mIA      mIB       VA       VB     mPtot    mIxhex   mIyhex   Vxhex    Vyhex  
     Voc3       0.00     0.00     0.00     1.431   -1.029   -0.402    0.00     0.00    -1.029   -1.431    0.00     0.00     0.00     1.296   -1.189  
     Isc3       2.82     -6.68    3.81     0.000    0.000   0.000     2.82     3.81     0.000    0.000    0.00     -0.70    -7.69    0.000    0.000  
    VrzIto      3.24     -3.24    0.00     1.427    0.000   -1.427    3.24     0.00     0.000   -1.427    0.00     2.29     -3.74    2.018    0.000  
    VztIro      0.00     -3.81    3.81     0.686   -1.021    0.335    0.00     3.81    -1.021   -0.686    2.62     -2.70    -4.40    0.249   -1.179  
    VtrIzo       nan      nan      nan      nan      nan      nan      nan      nan      nan      n